## Imports

In [67]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os
import geopandas as gpd

## Read-in

In [68]:
df = pd.read_csv('Miami_Beach_office_projects_popup_pin - Sheet1.csv')

In [69]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
project_gdf = gpd.read_file('Shvo\'s Alton Road office dev site.kml',driver = 'KML')
project_gdf['center_point'] = project_gdf.geometry.centroid

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24095/177959808.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  project_gdf['center_point'] = project_gdf.geometry.centroid


## Image URL collector

In [70]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [71]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

### Had to append an image because of png ###
links.append('https://s14.therealdeal.com/trd/m/up/2021/10/main-MIA-Location-Ventures-co-living_10.13.21-705x439.png')
df['image_links'] = links

In [72]:
links

['https://s13.therealdeal.com/trd/m/up/2022/10/Main_south-beach-project-705x439.jpg',
 'https://s11.therealdeal.com/trd/m/up/2022/10/Shvo-looks-to-redevelop-aging-Lincoln-Road-Clock-Tower-building-705x439.jpg',
 'https://s11.therealdeal.com/trd/m/up/2022/09/Main_shvo2-705x439.jpg',
 'https://s12.therealdeal.com/trd/m/up/2022/06/main_MIA_Shvo-NEW-705x439.jpg',
 'https://s13.therealdeal.com/trd/m/up/2022/01/Main_RHBH-ex-buys-7m-705x439.jpg',
 'https://s14.therealdeal.com/trd/m/up/2022/11/MMMIA-Wayne-Boich-Relateds-Bruce-Beal-and-SL-Green-CEO-plan-Miami-Beach-project-705x439.jpg',
 'https://s14.therealdeal.com/trd/m/up/2021/09/main-miami-jvdev-705x439.jpg',
 'https://s13.therealdeal.com/trd/m/up/2021/05/1200-Deco-Capital-moves-forward-with-new-version-of-long-planned-Sunset-Harbour-mixed-use-development-705x439.jpg',
 'https://s12.therealdeal.com/trd/m/up/2021/01/1200-Related-gets-height-increase-needed-for-Terminal-Island-office-project-in-Miami-Beach-705x439.jpg',
 'https://s14.thereald

https://s14.therealdeal.com/trd/m/up/2021/10/main-MIA-Location-Ventures-co-living_10.13.21-705x439.png

## Article link formatter

In [73]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2022/10...
1    <a href="https://therealdeal.com/miami/2022/10...
2    <a href="https://therealdeal.com/miami/2022/09...
3    <a href="https://therealdeal.com/miami/2022/06...
4    <a href="https://therealdeal.com/miami/2022/01...
5    <a href="https://therealdeal.com/miami/2022/11...
6    <a href="https://therealdeal.com/miami/2021/09...
7    <a href="https://therealdeal.com/miami/2021/05...
8    <a href="https://therealdeal.com/miami/2021/01...
9    <a href="https://therealdeal.com/miami/2021/10...
Name: description_link, dtype: object

## Google Maps API Geolocater Setup

In [74]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [75]:
df['geo_address'] = df['full_address'] + ' Miami Beach, FL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [76]:
df.columns

Index(['full_address', 'developers', 'project_name', 'sq_ft', 'description',
       'story_link', 'addtl_links', 'other_notes', 'image_links',
       'short_description', 'remaining_desc', 'description_link',
       'geo_address', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [77]:
df

,full_address,developers,project_name,sq_ft,description,story_link,addtl_links,other_notes,image_links,short_description,remaining_desc,description_link,geo_address,loc,point,lat,lon,altitude
0,944 Fifth Street and 411 Michigan Avenue,"Sumaida + Khurana, Bizzi & Partners and ex-Goo...",The Fifth Miami Beach,NaN,The Fifth Miami Beach is a partnership among S...,https://therealdeal.com/miami/2022/10/11/south...,https://therealdeal.com/miami/2022/06/16/prese...,(This is in the South of Fifth neighborhood in...,https://s13.therealdeal.com/trd/m/up/2022/10/M...,The Fifth Miami,Beach is a partnership among Sumaida + Khuran...,"<a href=""https://therealdeal.com/miami/2022/10...",944 Fifth Street and 411 Michigan Avenue Miami...,"(411 Michigan Ave, Miami Beach, FL 33139, USA,...","(25.7738297, -80.13782090000001, 0.0)",25.773830,-80.137821,0.0
1,407 Lincoln Road,Michael Shvo,NaN,NaN,"Shvo, which is Michael Shvo's company, wants t...",https://therealdeal.com/miami/2022/10/11/shvo-...,NaN,I currently have an inquiry out to find out wh...,https://s11.therealdeal.com/trd/m/up/2022/10/S...,"Shvo, which is","Michael Shvo's company, wants to redevelop th...","<a href=""https://therealdeal.com/miami/2022/10...","407 Lincoln Road Miami Beach, FL","(407 Lincoln Rd, Miami Beach, FL 33139, USA, (...","(25.7909259, -80.1327913, 0.0)",25.790926,-80.132791,0.0
2,1665-1667 Washington Avenue,Michael Shvo,NaN,21000,Shvo wants to build a six-story office buildin...,https://therealdeal.com/miami/2022/09/13/micha...,NaN,NaN,https://s11.therealdeal.com/trd/m/up/2022/09/M...,Shvo wants to build,"a six-story office building here with 21,000 ...","<a href=""https://therealdeal.com/miami/2022/09...","1665-1667 Washington Avenue Miami Beach, FL","(1665 Washington Ave, Miami Beach, FL 33139, U...","(25.7913384, -80.13162729999999, 0.0)",25.791338,-80.131627,0.0
3,1656 to 1680 Alton Road,Michael Shvo,NaN,"250,000","Shvo wants to develop a 250,000-square-foot of...",https://therealdeal.com/miami/2022/06/06/micha...,NaN,"Also, I created this map to show the exact pro...",https://s12.therealdeal.com/trd/m/up/2022/06/m...,Shvo wants to,"develop a 250,000-square-foot office project ...","<a href=""https://therealdeal.com/miami/2022/06...","1656 to 1680 Alton Road Miami Beach, FL","(1680 Alton Rd, Miami Beach, FL 33139, USA, (2...","(25.791536, -80.1416239, 0.0)",25.791536,-80.141624,0.0
4,653 Collins Avenue,Gebbia family office,NaN,"12,000",The Gebbia family will open the headquarters o...,https://therealdeal.com/miami/2022/01/03/famil...,NaN,(This is the Sunset Harbour neighborhood.),https://s13.therealdeal.com/trd/m/up/2022/01/M...,The Gebbia family,will open the headquarters of their Siebert F...,"<a href=""https://therealdeal.com/miami/2022/01...","653 Collins Avenue Miami Beach, FL","(653 Collins Ave, Miami Beach, FL 33139, USA, ...","(25.7764715, -80.13228149999999, 0.0)",25.776471,-80.132281,0.0
5,1920 Alton Road,"New York-based Related Companies, Miami Beach-...",NaN,"50,000","This is a partnership among: Bruce Beal, presi...",https://therealdeal.com/miami/2022/11/21/wayne...,NaN,NaN,https://s14.therealdeal.com/trd/m/up/2022/11/M...,This is a,"partnership among: Bruce Beal, president and ...","<a href=""https://therealdeal.com/miami/2022/11...","1920 Alton Road Miami Beach, FL","(1920 Alton Rd, Miami Beach, FL 33139, USA, (2...","(25.7996083, -80.1420236, 0.0)",25.799608,-80.142024,0.0
6,1501 Collins Avenue,Pebb Capital and Maxwelle Real Estate Group,NaN,NaN,Boca Raton-based Pebb Captial and Miami-based ...,https://therealdeal.com/miami/2021/09/07/jv-pl...,NaN,NaN,https://s14.therealdeal.com/trd/m/up/2021/09/m...,Boca Raton-based,Pebb Captial and Miami-based Maxwelle Real Es...,"<a href=""https://therealdeal.com/miami/2021/09...","1501 Collins Avenue Miami Beach, FL","(1501 Collins Ave, Miami Beach, FL 33139, USA,...","(25.787969, -80.1298389, 0.0)",25.787969,-80.129839,0.0
7,1759 Purdy Avenue,Bradley Colmer's Deco Capital Group and New Yo...,Eighteen Sunset,"

## Correction section

In [78]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

### Change value to percent value
### df['percent_change'] = pd.Series(['{0:.2f}%'.format(val) for val in df['percent_change']], index = df.index)

## HTML popup formatter

In [79]:
df.columns

Index(['full_address', 'developers', 'project_name', 'sq_ft', 'description',
       'story_link', 'addtl_links', 'other_notes', 'image_links',
       'short_description', 'remaining_desc', 'description_link',
       'geo_address', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [80]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    developers = df['developers'].iloc[i]
    image = df['image_links'].iloc[i]
    description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developers: <strong>{}'''.format(developers) + '''</strong><br>
    <strong>{}'''.format(description) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [81]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [82]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/10/11/south-beach-office-project-backed-by-google-ex-ceo-eric-schmidt-launches-leasing/" target="_blank" rel="noopener noreferrer">The Fifth Miami</a> Beach is a partnership among Sumaida + Khurana and Bizzi & Partners, both based in New York, as well as ex-Google CEO Eric Schmidt who together with his wife has an 88 percent stake in the entity that owns the developments sites. The two New York-based firms have a 12 percent stake. It\'s a five-story office building with 3,125 square feet of ground-floor retail'

In [83]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=13)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Office Projects in Miami Beach")

for i, r in project_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j)
    geo_j.add_to(m)

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [84]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [85]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [86]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/miami_beach_office_projects
